In [16]:
# install needed stuff
!pip install tensorflow_decision_forests

In [17]:
# imports
import pandas as pd
import numpy as np
import time
from random import sample
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import tensorflow_decision_forests as tfdf

# seed
np.random.seed(0)

In [18]:
# where data is located
!ls ./drive/MyDrive/Thesis\ Workspace/Notebooks/data/set-3
DATA_PATH = "./drive/MyDrive/Thesis Workspace/Notebooks/data/sb14-set3/"

dev  test  train


In [19]:
# load user data
train = pd.read_csv(DATA_PATH + 'train.csv')
val = pd.read_csv(DATA_PATH + 'test.csv')
print('Shapes', train.shape, val.shape)
train.head()

Shapes (8278, 8) (1183, 8)


,statuses_count,favourites_count,listed_count,label,age,follower_to_friend_ratio,tweet_frequency,url_count
0,9798,142,45568,0,13.127071,22181.497110,746.396501,207
1,0,7,0,1,1.365350,0.000000,0.000000,0
2,5518,953,3201,0,13.101479,1605.976842,421.173819,95
3,192876,2946,1744,0,13.411272,68.233076,14381.633489,173
4,103,729,44,1,8.402246,20.593509,12.258627,57


In [20]:
# Convert the dataset into a TensorFlow dataset.
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label="label")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(val, label="label")

In [21]:
# Train a Random Forest model.
model = tfdf.keras.RandomForestModel()
model.fit(train_ds)

Use /tmp/tmp3274jytw as temporary training directory
Starting reading the dataset
8/9 [=========================>....] - ETA: 0s
Dataset read in 0:00:00.319504
Training model
Model trained in 0:00:09.023425
Compiling model
9/9 [==============================] - 11s 1s/step


In [22]:
# Summary of the model structure.
model.summary()

Model: "random_forest_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (7):
	age
	favourites_count
	follower_to_friend_ratio
	listed_count
	statuses_count
	tweet_frequency
	url_count

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1.                  "__LABEL" 11.441025 ################
    2.          "tweet_frequency"  7.303045 #########
    3.           "statuses_count"  7.302433 #########
    4.         "favourites_count"  6.786136 ########
    5.                      "age"  5.244687 ######
    6.                "url_count"  4.189694 ####
    7.             "listed_count"  2.558781 ##
    8. "follower_to_friend_ratio"  0.953464 

Variable Importance: NUM_AS_ROOT:
    1. "follower_to_frie

In [23]:
# Evaluate the model.
model.compile(metrics=['accuracy','AUC','Recall','Precision'])
print(model.evaluate(test_ds,return_dict=True))

2/2 [==============================] - 1s 14ms/step - loss: 0.0000e+00 - accuracy: 0.7591 - auc: 0.8287 - recall: 0.8516 - precision: 0.7415
{'loss': 0.0, 'accuracy': 0.759087085723877, 'auc': 0.8287278413772583, 'recall': 0.8515625, 'precision': 0.7414966225624084}


In [24]:
pred = model.predict(test_ds) > 0.5
print(classification_report(pred, val['label'], digits=4))
print('ROC acc: ',roc_auc_score(pred, val['label']))

              precision    recall  f1-score   support

       False     0.6501    0.7879    0.7124       448
        True     0.8516    0.7415    0.7927       735

    accuracy                         0.7591      1183
   macro avg     0.7508    0.7647    0.7526      1183
weighted avg     0.7753    0.7591    0.7623      1183

ROC acc:  0.7647215136054423
